In [ ]:
import win32com.client
import os
import re
import pandas as pd
import datetime as dt
import time

from std_naming import *

In [ ]:

### Root Export Folder and Dictionary Directory
root_export_dir = 'D:/outlook_reports/raw_emails'
dictionary_dir = 'd:/git/dictionaries'
# Connect to Outlook Worker
import win32com.client

In [ ]:

outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

In [ ]:
# Pull all Folders in gmcwilliams@example.co Email Account
for folders in outlook.Folders:
    print(folders)

In [ ]:
account_reports_manager = outlook.Folders['Reports Manager']
for folders in account_reports_manager.Folders:
    print(folders)

In [ ]:
## Assign all folders variables for iteration later
inbox = account_reports_manager.Folders['Inbox']
sent = account_reports_manager.Folders['Sent Items']
outbox = account_reports_manager.Folders['Outbox']
files = account_reports_manager.Folders['Files']
drafts = account_reports_manager.Folders['Drafts']
deleted = account_reports_manager.Folders['Deleted Items']
junk = account_reports_manager.Folders['Junk Email']

In [ ]:
for folder in inbox.Folders:
    print(folder)

In [ ]:
# Create DataFrame from Reports Manager Email Account
## Create Initial DataFrame from Inbox
emails_list = []

#for folder in reports_junk.Folders:

messages = inbox.Items
messages.Sort('[ReceivedTime]', True)

In [ ]:
parser = NameShaping(dictionary_source=os.getcwd())

In [ ]:

i = 0

for message in messages:
    # print(message)
    message_dict = {}
    try:
        message_dict['from'] = message.SenderEmailAddress
    except:
        pass
    try:
        message_dict['to'] = message.To
    except:
        pass
    try:
        message_dict['date'] = message.ReceivedTime.strftime("%Y-%m-%d %H:%M:%S")
    except:
        pass
    try:
        message_dict['subject'] = message.Subject
    except:
        pass
    try:

        message_dict["std_name"] = parser.std_client_names_rename(message.Subject)
        print(message.Subject)
        print(message_dict["std_name"])
    except:
        pass

    try:
        message_dict['body'] = message.Body
    except:
        pass
    attachments_list = []
    for attachment in message.Attachments:
        try:
            attachment_name = str(attachment).lower().replace(" ", "_").replace("-", "_")
            attachments_list.append(attachment_name)
            # attachment.SaveASFile(path+ '\\' + attachment_name)
        except Exception as e:
            print(e)

    message_dict['attachments'] = attachments_list
    message_dict['messageObject'] = message
    emails_list.append(message_dict)

    i = i + 1
    if i >= 1000:
        break



In [ ]:
df = pd.DataFrame(emails_list)

In [ ]:
df

In [ ]:
def save_email_to_folder(subject, message_obj):
    try:
        message_obj.SaveAs(f'{root_export_dir}/{subject}.eml')
    except Exception as e:
        print(e)


In [ ]:
df[["subject","messageObject"]].apply(lambda x: save_email_to_folder(x['subject'], x['messageObject']), axis=1)

In [ ]:
df

In [ ]:
print(os.getcwd())

In [ ]:
df = pd.read_parquet(f"{os.getcwd()}/standard_client_names.parquet")

In [ ]:
df.to_csv(f".csv", index=False)

In [ ]:
df